In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from FinishedCode.importData import ImportEV
from FinishedCode.dataTransformation import createUser
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [42]:
start, end = "2018-06-01", "2018-11-09"
df_train = ImportEV().getCaltech(start_date=start, end_date=end, removeUsers=True, userSampleLimit=25)

,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
3074,5bc92374f9af8b0dc677c276,0039,2018-06-18 07:33:04+00:00,2018-06-18 17:53:28+00:00,2018-06-18 17:22:40+00:00,19.900,2_39_138_29_2018-06-18 14:33:03.983186,0002,CA-304,2-39-138-29,America/Los_Angeles,000000061,"[{'WhPerMile': 250, 'kWhRequested': 5.0, 'mile..."
3143,5bc923c7f9af8b0dc677c2bb,0039,2018-06-19 07:30:27+00:00,2018-06-19 17:40:44+00:00,2018-06-19 14:44:19+00:00,24.048,2_39_138_29_2018-06-19 14:30:26.698782,0002,CA-304,2-39-138-29,America/Los_Angeles,000000061,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
3941,5bc92770f9af8b0dc677c5d9,0039,2018-07-02 06:52:23+00:00,2018-07-02 16:05:39+00:00,2018-07-02 12:25:06+00:00,10.411,2_39_79_380_2018-07-02 13:52:22.922710,0002,CA-489,2-39-79-380,America/Los_Angeles,000000066,"[{'WhPerMile': 400, 'kWhRequested': 28.0, 'mil..."
4367,5bc92926f9af8b0dc677c783,0039,2018-07-09 13:07:37+00:00,2018-07-09 19:08:23+00:00,2018-07-09 15:46:45+00:00,17.077,2_39_123_557_2018-07-09 20:07:36.746469,0002,CA-503,2-39-123-557,America/Los_Angeles,000000022,"[{'WhPerMile': 350, 'kWhRequested': 21.0, 'mil..."
5923,5bc92f69f9af8b0dc677cd97,0039,2018-07-31 16:54:07+00:00,2018-07-31 18:36:47+00:00,2018-07-31 18:36:34+00:00,8.928,2_39_131_30_2018-07-31 23:54:07.294444,0002,CA-305,2-39-131-30,America/Los_Angeles,000000216,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13462,5c2e86e4f9af8b13dab0763b,0039,2018-11-08 08:05:51+00:00,2018-11-08 12:40:50+00:00,2018-11-08 09:21:49+00:00,5.478,2_39_79_380_2018-11-08 15:05:50.914103,0002,CA-489,2-39-79-380,America/Los_Angeles,000000066,"[{'WhPerMile': 313, 'kWhRequested': 12.52, 'mi..."
13466,5c2e86e4f9af8b13dab0763f,0039,2018-11-08 08:50:18+00:00,2018-11-08 17:48:58+00:00,2018-11-08 14:00:52+00:00,21.283,2_39_79_376_2018-11-08 15:50:17.561029,0002,CA-324,2-39-79-376,America/Los_Angeles,000000234,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
13479,5c2e86e4f9af8b13dab0764c,0039,2018-11-08 09:41:48+00:00,2018-11-08 18:32:10+00:00,2018-11-08 11:41:28+00:00,6.717,2_39_138_29_2018-11-08 16:41:47.968540,0002,CA-304,2-39-138-29,America/Los_Angeles,000000559,"[{'WhPerMile': 313, 'kWhRequested': 9.39, 'mil..."
13480,5c2e86e4f9af8b13dab0764d,0039,2018-11-08 09:46:08+00:00,2018-11-08 20:54:21+00:00,2018-11-08 10:44:59+00:00,1.898,2_39_92_436_2018-11-08 16:46:08.414528,0002,CA-318,2-39-92-436,America/Los_Angeles,000000324,"[{'WhPerMile': 400, 'kWhRequested': 6.0, 'mile..."


In [43]:
df[df['userID'] == '000000562']


,_id,clusterID,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
10172,5bc94053f9af8b0dc677de30,0039,2018-09-24 12:57:01+00:00,2018-09-24 17:41:43+00:00,2018-09-24 13:48:34+00:00,1.464000,2_39_79_378_2018-09-24 19:57:00.517293,0002,CA-326,2-39-79-378,America/Los_Angeles,000000562,"[{'WhPerMile': 333, 'kWhRequested': 6.66, 'mil..."
10237,5bc940b4f9af8b0dc677de71,0039,2018-09-25 08:22:59+00:00,2018-09-25 12:06:01+00:00,2018-09-25 10:34:59+00:00,4.647000,2_39_131_30_2018-09-25 15:22:58.731565,0002,CA-305,2-39-131-30,America/Los_Angeles,000000562,"[{'WhPerMile': 333, 'kWhRequested': 6.66, 'mil..."
10273,5bc940b4f9af8b0dc677de95,0039,2018-09-25 13:07:08+00:00,2018-09-25 19:52:16+00:00,2018-09-25 13:46:48+00:00,2.492660,2_39_78_361_2018-09-25 20:07:07.556648,0002,CA-493,2-39-78-361,America/Los_Angeles,000000562,"[{'WhPerMile': 227, 'kWhRequested': 2.27, 'mil..."
10346,5bc94120f9af8b0dc677dede,0039,2018-09-26 08:53:08+00:00,2018-09-26 12:56:29+00:00,2018-09-26 11:10:40+00:00,4.434000,2_39_124_22_2018-09-26 15:53:08.396167,0002,CA-312,2-39-124-22,America/Los_Angeles,000000562,"[{'WhPerMile': 227, 'kWhRequested': 6.81, 'mil..."
10440,5bc94185f9af8b0dc677df3c,0039,2018-09-27 08:49:04+00:00,2018-09-27 18:52:55+00:00,2018-09-27 11:01:12+00:00,4.744000,2_39_124_22_2018-09-27 15:49:04.309550,0002,CA-312,2-39-124-22,America/Los_Angeles,000000562,"[{'WhPerMile': 227, 'kWhRequested': 6.81, 'mil..."
10531,5bc941ddf9af8b0dc677df97,0039,2018-09-28 08:46:15+00:00,2018-09-28 18:21:16+00:00,2018-09-28 10:50:46+00:00,5.370000,2_39_124_22_2018-09-28 15:46:15.424408,0002,CA-312,2-39-124-22,America/Los_Angeles,000000562,"[{'WhPerMile': 227, 'kWhRequested': 6.81, 'mil..."
10726,5be2fafef9af8b2b0edf9ea7,39,2018-10-01 08:48:12+00:00,2018-10-01 12:48:54+00:00,2018-10-01 11:09:12+00:00,4.906000,2_39_89_439_2018-10-01 15:48:11.826083,2,CA-501,2-39-89-439,America/Los_Angeles,000000562,"[{'WhPerMile': 227, 'kWhRequested': 6.81, 'mil..."
10751,5be2fafef9af8b2b0edf9ec0,39,2018-10-01 13:50:50+00:00,2018-10-01 18:01:13+00:00,2018-10-01 14:36:32+00:00,1.572000,2_39_95_27_2018-10-01 20:50:49.827112,2,CA-319,2-39-95-27,America/Los_Angeles,000000562,"[{'WhPerMile': 227, 'kWhRequested': 4.54, 'mil..."
10811,5be2fb55f9af8b2b0edf9efc,39,2018-10-02 08:43:04+00:00,2018-10-02 12:33:53+00:00,2018-10-02 10:31:37+00:00,5.025000,2_39_123_23_2018-10-02 15:43:04.061352,2,CA-313,2-39-123-23,America/Los_Angeles,000000562,"[{'WhPerMile': 227, 'kWhRequested': 4.54, 'mil..."
10913,5be2fba6f9af8b2b0edf9f62,39,2018-10-03 08:37:38+00:00,2018-10-03 19:11:58+00:00,2018-10-03 10:20:25+00:00,4.880000,2_39_138_29_2018-10-03 15:37:38.338676,2,CA-304,2-39-138-29,America/Los_Angeles,000000562,"[{'WhPerMile': 227, 'kWhRequested': 6.81, 'mil..."


In [17]:
users = createUser(df_train, start, end)
userID = Users.data.userID.unique()
User_df = []

for user in userID:
    User_df.append(users.getUserData(user=user))

print(User_df[1])

,chargingTime,kWhDelivered,Year,Month,Day,Weekday
0,0.0,0.0,2018,06,01,4
1,0.0,0.0,2018,06,02,5
2,0.0,0.0,2018,06,03,6
3,0.0,0.0,2018,06,04,0
4,0.0,0.0,2018,06,05,1
...,...,...,...,...,...,...
156,0.0,0.0,2018,11,04,6
157,0.0,0.0,2018,11,05,0
158,0.0,0.0,2018,11,06,1
159,0.0,0.0,2018,11,07,2


In [ ]:


df.userID.duplicated()

In [26]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1))
df_scaler = sc.fit_transform(user_61[['chargingTime', 'kWhDelivered']])

In [28]:
x = user_61.drop(columns = ['kWhDelivered']).values
y = user_61['kWhDelivered'].values
x

array([[0.0, '2018', '06', '01', 4],
       [0.0, '2018', '06', '02', 5],
       [0.0, '2018', '06', '03', 6],
       [0.0, '2018', '06', '04', 0],
       [0.0, '2018', '06', '05', 1],
       [0.0, '2018', '06', '06', 2],
       [0.0, '2018', '06', '07', 3],
       [0.0, '2018', '06', '08', 4],
       [0.0, '2018', '06', '09', 5],
       [0.0, '2018', '06', '10', 6],
       [0.0, '2018', '06', '11', 0],
       [0.0, '2018', '06', '12', 1],
       [0.0, '2018', '06', '13', 2],
       [0.0, '2018', '06', '14', 3],
       [0.0, '2018', '06', '15', 4],
       [0.0, '2018', '06', '16', 5],
       [0.0, '2018', '06', '17', 6],
       [35376.0, '2018', '06', '18', 0],
       [26032.0, '2018', '06', '19', 1],
       [0.0, '2018', '06', '20', 2],
       [0.0, '2018', '06', '21', 3],
       [0.0, '2018', '06', '22', 4],
       [0.0, '2018', '06', '23', 5],
       [0.0, '2018', '06', '24', 6],
       [0.0, '2018', '06', '25', 0],
       [0.0, '2018', '06', '26', 1],
       [0.0, '2018', '06', '27

In [20]:
x_train, x_rem, y_train, y_rem = train_test_split(x, y, train_size=0.8)

In [21]:
x_valid, x_test, y_valid, y_test = train_test_split(x_rem, y_rem, test_size=0.5)

In [29]:
 print(x.shape), print(y.shape)

(161, 5)
(161,)


(None, None)

In [36]:
#reshape

x = x.reshape((x.shape[0], x.shape[1], 1))
#x.value.reshape(-1,1)
print(x.shape)

(161, 5, 1)


In [39]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(5,1))) # todo: add inputs steps
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')



In [40]:
model.fit(x_train, y_train, epochs=100, verbose=0)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).